# Relevant imports

In [ ]:
import pennylane as qml
import numpy as np
import tensorflow as tf

# Provided formatting functions

In [ ]:
def array_to_concatenated_string(array):
    return ",".join(str(x) for x in array)

def concatenated_string_to_array(string):
    return np.array([int(x) for x in string.split(",")])

def parse_input(giant_string):
    X_train_part, Y_train_part, X_test_part = giant_string.split("XXX")

    X_train_row_strings = X_train_part.split("S")
    X_train_rows = [[float(x) for x in row.split(",")] for row in X_train_row_strings]
    X_train = np.array(X_train_rows)

    Y_train = concatenated_string_to_array(Y_train_part)

    X_test_row_strings = X_test_part.split("S")
    X_test_rows = [[float(x) for x in row.split(",")] for row in X_test_row_strings]
    X_test = np.array(X_test_rows)

    return X_train, Y_train, X_test

# Reading input, building train/test sets

In [ ]:
#All data read from input.txt file, one long string
text_file = open("input.txt", "r")
data = text_file.read()
text_file.close()
X_train, Y_train, X_test = parse_input(data)

#format Y_train data to One Hot encoding
Y_train = Y_train + 1
Y_train = tf.one_hot(Y_train, 3)

#Y_test is from Challenge github. It's only neccessary for the judging demo to show accuracy
Y_test = tf.one_hot(np.array([1,0,-1,0,-1,1,-1,-1,0,-1,1,-1,0,1,0,-1,-1,0,0,1,1,0,-1,0,0,-1,0,-1,0,0,1,1,-1,-1,-1,0,-1,0,1,0,-1,1,1,0,-1,-1,-1,-1,0,0
])+1, 3)

# Building quantum circuit (3 Qubits)

In [ ]:
n_neurons = 3
device = qml.device("default.qubit", wires=n_neurons)

@qml.qnode(device)
def qnode(inputs, weights):
    
    #Encode n_layers features into QuBits 
    qml.AngleEmbedding(inputs, wires=range(n_neurons))
    
    #One paramater rotation layer with CNOT gates on each QuBit
    qml.BasicEntanglerLayers(weights, wires=range(n_neurons))
    
    #Returns evaluation of PauliZ matrix on each QuBit feature
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_neurons)]

# Building quantum layer and Tensorflow model

In [ ]:
n_layers = 3
weight_shapes = {"weights": (n_layers, n_neurons)}
#Convert QNode into Keras Layer
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_neurons)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(3, activation='relu'))
model.add(qlayer)
model.add(tf.keras.layers.Dense(3, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.09)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

# Fitting model to trainset

In [ ]:
model.fit(X_train, Y_train, epochs=2, batch_size=5, verbose=1)

# Evaluating model with unseen test data

In [ ]:
#Evaluating Model
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))

# Formatting test results 

In [ ]:
predictions = model.predict(X_test, verbose=0)
output = (tf.argmax(predictions, axis=1)-1).numpy()
output = array_to_concatenated_string(output)
print(output)